In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm.auto import tqdm
import random

In [6]:
NUM_PAGES = 88
ROOT = "https://codeforces.com"
PAGE = ROOT + "/problemset/page/"

links = []
tags = []

for page_index in tqdm(range(1, NUM_PAGES+1)):
    webpage = BeautifulSoup(requests.get(PAGE+str(page_index)).text, "lxml")
    table = webpage.find("table", class_="problems")
    table_rows = table.find_all("tr")[1:]
    for row in table_rows:
        table_cell = row.find_all("td")[1]
        a_tags = table_cell.find_all("a")
        links.append(ROOT + a_tags[0]["href"])
        temp_tags = []
        for tag in a_tags[1:]:
            temp_tags.append(tag.text)
        tags.append(temp_tags)

  0%|          | 0/88 [00:00<?, ?it/s]

In [7]:
print(len(set(links)), len(tags))

8741 8741


* 'le' -> '<='
* '$' -> ''
* 'ge' -> '>='
* 'times' -> '*'
* '^{\\dagger}' -> ''
* '\\ldots' -> '...'
* '\xa0' -> ''
* '\\dots' -> ''
* '\\ne' -> '!='

In [26]:
qns = []
ios = []
true_tags = []
for i, link in tqdm(enumerate(links)):
    try:
        content = BeautifulSoup(requests.get(link).text, "lxml")
        cp = content.find("div", class_="problem-statement")
        div_10 = cp.find_all("div")[10]
        qn = div_10.text.replace('^{\\dagger}', '').replace('$', '').replace("\\ldots", '...').replace("\\le", "<=").replace("\\ge", ">=").replace("\\times", '*')\
                .replace("\\dots", '...').replace("\xa0", ' ').replace(" \\ne ", '!=').replace("\\dagger", '')
        io = "Input: " + str(cp.find("div", class_="input").find("pre").text.split("\n")) + \
             " Output:" + str(cp.find("div", class_="output").find("pre").text.split("\n"))
        qns.append(qn)
        ios.append(io)
        true_tags.append(tags[i])
    except:
        pass

0it [00:00, ?it/s]

In [28]:
len(qns), len(ios), len(true_tags)

(8564, 8564, 8564)

In [30]:
df = pd.DataFrame(data={"question_text":qns, "input_outputs":ios, "tags":true_tags})
df.to_csv("../datasets/codeforces.csv")